## 加载环境变量

In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

## 字符串模板

In [2]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "你是一个{name}，帮我起一个具有{county}特色的{sex}名字"
)
prompt = prompt_template.format(
    name="算命大师", county="法国", sex="男孩"
)  # 使用 format 方法填充模板中的变量

print(prompt)

你是一个算命大师，帮我起一个具有法国特色的男孩名字


## 对话模板

In [4]:
from langchain_core.prompts import ChatPromptTemplate

# 使用对话模板模拟人机对话
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师，你的名字叫{name}"),
        ("human", "你好{name}，你感觉如何？"),
        ("ai", "你好，我的状态非常好！"),
        ("human", "你叫什么名字"),
        ("ai", "你好！我叫{name}"),
        ("human", "{user_input}"),
    ]
)

message = chat_template.format_messages(
    name="算命大师", user_input="你的名字叫什么？"
)  # 使用 format_messages 方法填充模板中的变量

print(message)

[SystemMessage(content='你是一个起名大师，你的名字叫算命大师', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好算命大师，你感觉如何？', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，我的状态非常好！', additional_kwargs={}, response_metadata={}), HumanMessage(content='你叫什么名字', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！我叫算命大师', additional_kwargs={}, response_metadata={}), HumanMessage(content='你的名字叫什么？', additional_kwargs={}, response_metadata={})]


## 消息占位符

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage


chat_template = ChatPromptTemplate.from_messages(
    [("system", "你是一个很厉害的人工智能助手"), MessagesPlaceholder("msgs")]
)

# 此写法与上面效果一致
# chat_template = ChatPromptTemplate.from_messages(
#     [("system", "你是一个很厉害的人工智能助手"), ("placeholder", "{msgs}")]
# )

# 消息占位符可以创建Chain之后仍然可以插入消息
result = chat_template.invoke({"msgs": [HumanMessage(content="hi!")]})
print(result)

messages=[SystemMessage(content='你是一个很厉害的人工智能助手', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]


## 组合模板
- 灵活性高

In [10]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# SystemMessage 系统提示词
sy = SystemMessage(content="你是一个起名大师", additional_kwargs={"大师名字": "陈瞎子"})

# HumanMessage 用户的输入
hu = HumanMessage(content="请问大师叫什么？")

# AIMessage AI的输出
ai = AIMessage(content="你好，我的名字叫陈瞎子")


# 使用消息拼成模板
message = [sy, hu, ai]
print(message)


[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师名字': '陈瞎子'}, response_metadata={}), HumanMessage(content='请问大师叫什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，我的名字叫陈瞎子', additional_kwargs={}, response_metadata={})]


## 自定义提示词模板

In [17]:
# 函数大师：可以根据函数的名称，查找函数代码，并给我中文的代码说明。
from langchain_core.prompts import StringPromptTemplate


# 定义一个简单的函数作为示例效果
def hello_world(abc):
    print("hello, world!")
    return abc


PROMPT_TEMPLATE = """
你是一个非常有经验和天赋的程序员，现在给你如下函数名，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称：{function_name}
源代码：
{source_code}
代码解释：
"""


def get_source_code(function_name):
    import inspect

    source_code = inspect.getsource(function_name)
    return source_code


# 自定义的提示词模板
class CustomPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        # 获取源代码
        source_code = get_source_code(kwargs["function_name"])

        # 生成提示词模板
        prompt = PROMPT_TEMPLATE.format(
            function_name=kwargs["function_name"].__name__,
            source_code=source_code,
        )
        return prompt


# 使用自定义的提示词模板
custom_prompt_template = CustomPromptTemplate(input_variables=["function_name"])
prompt_value = custom_prompt_template.format(function_name=hello_world)
print(prompt_value)


你是一个非常有经验和天赋的程序员，现在给你如下函数名，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称：hello_world
源代码：
def hello_world(abc):
    print("hello, world!")
    return abc

代码解释：

